# StableDiffusionXL + ControlNetのサンプル利用コード
##利用方法
・基本的には全てのセルを実行することで利用できます。

・GPUを利用しましょう。（利用制限があるのでこまめにインスタンスは削除してください）

（変更する際は，「ランタイム」→「ランタイムのタイプを変更」からGPUを選択してください）

##注意点
・本ファイルの相対的な場所は変更しないでください。

・本ファイルの名前を変更しないでください。

変更する場合は，[2]セル目のファイル名も一緒に変更してください。

・[1]セル目の実行時にセッションの再起動を求められることがあります。

その場合はポップアップに従って再起動したあと，全てのセルを再度実行してください。

In [1]:
#SDXL で必要なモジュールのインストール
!pip install -U peft tensorflow-metadata diffusers transformers scikit-learn ftfy accelerate invisible_watermark safetensors controlnet-aux mediapipe timm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mediapipe to determine which version is compatible with other requirements. This could take a while.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12

下記のコードを実行すると，認証が求められるので，認証してください。（Google colabにマイドライブの中を読めるようにする認証です）

In [2]:
#Google Driveのフォルダをマウント（認証入る）
from google.colab import drive
drive.mount('/content/drive')

# カレントディレクトリを本ファイルが存在するディレクトリに変更する。
import glob
import os
pwd = os.path.dirname(glob.glob('/content/drive/MyDrive/colabzenn/**/colab_SDXL_LCMLoRA_sample/SDXLLCMLoRA_sample.ipynb', recursive=True)[0])
print(pwd)

%cd $pwd
!pwd

Mounted at /content/drive
/content/drive/MyDrive/colabzenn/colab_SDXL_LCMLoRA_sample
/content/drive/MyDrive/colabzenn/colab_SDXL_LCMLoRA_sample
/content/drive/MyDrive/colabzenn/colab_SDXL_LCMLoRA_sample


In [3]:
#モジュールをimportする
from module.module_sdc import SDXLC
import time

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

モデルを変更する場合は，下記のconfigを変更してください。

「;」を付けるとコメントアウトできます。


In [4]:
#モデルの設定を行う。

#sdxlのモデルはvariant = "fp16"で読み込んでいるため、モデルのsafetensor名にfp16と入ってるモデルを利用してください。
#from_single_file = Trueでsafetensor単体のモデルも読み込めるようになりました。
#その場合は、base_model_pathにsafetensorのpathを指定してください。

config_text = """
[SDXLC]
device = auto
n_steps=4
high_noise_frac=None
seed=42

;from_single_file = True
vae_model_path = None
base_model_path = Asahina2K/Animagine-xl-3.1-diffuser-variant-fp16
;base_model_path = ./inputs/fudukiMix_v20.safetensors
refiner_model_path = None

;controlnet_path = xinsir/controlnet-openpose-sdxl-1.0
;controlnet_path = diffusers/controlnet-canny-sdxl-1.0
controlnet_path = diffusers/controlnet-depth-sdxl-1.0
;controlnet_path = diffusers/controlnet-zoe-depth-sdxl-1.0
;controlnet_path = xinsir/controlnet-tile-sdxl-1.0
;controlnet_path = xinsir/controlnet-union-sdxl-1.0
;controlnet_path = xinsir/controlnet-scribble-sdxl-1.0

;control_mode = openpose
;control_mode = openpose_face
;control_mode = openpose_faceonly
;control_mode = openpose_full
;control_mode = canny
control_mode = depth
;control_mode = zoe_depth
;control_mode = tile
;control_mode = scribble

;lora_weight_path = ./inputs/DreamyvibesartstyleSDXL.safetensors
;lora_weight_path = latent-consistency/lcm-lora-sdxl
lora_weight_path = ./inputs/lcm-animaginexl-3_1.safetensors
;lora_weight_path = None
lora_scale = 1.0

select_solver = LCM
;select_solver = DPM
;select_solver = Eulera

use_karras_sigmas = True
scheduler_algorithm_type = dpmsolver++
solver_order = 2

cfg_scale = 2.0
width = 832
height = 1216
;width = 1024
;height = 1024
output_type = pil
aesthetic_score = 6
negative_aesthetic_score = 2.5

save_latent_simple = False
save_latent_overstep = False
save_latent_approximation = False

"""

with open("configs/config.ini", "w", encoding="utf-8") as f:
  f.write(config_text)

生成する画像を決定するプロンプトを指定します。

negative_promptを設定することで，ここに書かれている内容を回避できる。


In [5]:
#読み上げるプロンプトを設定する。

main_prompt = """
1 girl ,Yellowish-white hair ,short hair ,red small ribbon,red eyes,red hat ,school uniform ,solo ,smile ,upper body ,Anime ,Japanese,best quality,high quality,ultra highres,ultra quality
"""

#main_prompt = """
#a cute girl, lips, Facing forward, looking straight at me, anime, pixiv, masterpiece,  best quality
#"""

"""use fudukiMix_v20"""
#main_prompt = """
#japanese woman, (close-up:2), (natural lighting),wavy hair, forehead, (dark brown eyes),(from side), (downward slanting eyes),pouty, (white sweater),(dyanmic posing) (see-through curtain, bright room)
#"""

use_lora = False
if use_lora:
  main_prompt += ", Dreamyvibes Artstyle"


negative_prompt="""
nsfw, lowres, (bad), text, error, fewer, extra, missing, worst quality, jpeg artifacts, low quality, watermark, unfinished, displeasing, oldest, early, chromatic aberration, signature, extra digits, artistic error, username, scan, [abstract]
"""

#negative_prompt = ""

"""use fudukiMix_v20"""
#negative_prompt="""
#(cleavage:2), (illustration), 3d, 2d, painting, cartoons, sketch, watercolor, monotone,(kimono), (crossed eyes), (strabismus)
#"""

#参照画像を"./inputs/refer_tate.png"に格納してから実行する。
input_refer_image_path = "./inputs/refer_tate.png"
output_refer_image_path = "./inputs/refer_c.png"





In [6]:

sd = SDXLC()
#指定しているControlNetで利用できる"./inputs/refer_c.png"がすでにある場合は、下記はコメントアウトしても良い
sd.prepare_referimage(input_refer_image_path = input_refer_image_path, output_refer_image_path = output_refer_image_path, low_threshold = 100, high_threshold = 200)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

The config attributes {'skip_prk_steps': True} were passed to LCMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


dpt_hybrid-midas-501f0c75.pt:   0%|          | 0.00/493M [00:00<?, ?B/s]

In [7]:
for i in range(5):
      start = time.time()
      image = sd.generate_image(main_prompt, neg_prompt = negative_prompt,image_path = output_refer_image_path, controlnet_conditioning_scale = 0.7)
      print("generate image time: ", time.time()-start)
      image.save("./outputs/SDXLC_result_{}.png".format(i))

  0%|          | 0/4 [00:00<?, ?it/s]

generate image time:  8.840204238891602


  0%|          | 0/4 [00:00<?, ?it/s]

generate image time:  6.226426839828491


  0%|          | 0/4 [00:00<?, ?it/s]

generate image time:  6.1514317989349365


  0%|          | 0/4 [00:00<?, ?it/s]

generate image time:  6.300405025482178


  0%|          | 0/4 [00:00<?, ?it/s]

generate image time:  6.286590814590454
